# Tutorial 2: Types as Boundaries

*Base Types and Arrow Types*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-typed-passages/blob/main/notebooks/02-types-as-boundaries.ipynb)

---

## A Note from the Archives

> *Year 802, Capital Archives*
>
> "Passages must declare what they accept."
>
> Brennis Mund stood before the Classification Integrity Committee, presenting his radical proposal. "A passage that accepts natural numbers should be marked as such. A passage that accepts booleans should be marked as such. And crucially — a passage should **never** be permitted to accept itself."
>
> The committee was skeptical. "But the Pure Passage Calculus has no such distinctions," objected Elder Archivist Kellum. "Any passage can be applied to any passage."
>
> "That," said Brennis, "is precisely the problem."

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Define base types (nat, bool)
2. Define arrow types (τ₁ → τ₂)
3. Assign types to simple lambda expressions
4. Explain why types prevent self-application

## Setup

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions_df = pd.read_csv(BASE_URL + "typed_passage_expressions.csv")
comparison_df = pd.read_csv(BASE_URL + "type_system_comparison.csv")

print(f"Loaded {len(expressions_df)} expressions")

## 1. The Problem with Untyped Passages

In untyped lambda calculus, every expression can be applied to every other expression:

- $(\lambda x.x)\ 5$ — identity applied to a number ✓
- $(\lambda x.x)\ (\lambda y.y)$ — identity applied to identity ✓
- $(\lambda x.x\ x)\ (\lambda x.x\ x)$ — self-application ✓ (but loops forever!)

There's no way to distinguish "safe" applications from "dangerous" ones.

In [ ]:
# Compare typable vs untypable expressions
typable = expressions_df[expressions_df['typable'] == True]
untypable = expressions_df[expressions_df['typable'] == False]

print(f"Typable expressions: {len(typable)}")
print(f"Untypable expressions: {len(untypable)}")

print("\nUntypable expressions (all involve self-application):")
for _, row in untypable.iterrows():
    print(f"  {row['expression'][:50]}...")

## 2. Base Types: The Atoms

Brennis proposed starting with **base types** — atomic classifications that cannot be broken down:

| Base Type | Notation | Meaning |
|-----------|----------|--------|
| Natural numbers | `nat` | Church numerals: 0, 1, 2, ... |
| Booleans | `bool` | Church booleans: TRUE, FALSE |

These correspond to Kelleth Mund's Church encodings from Course 1.

In [ ]:
# Church encodings and their types
print("Church Numerals:")
print("  ZERO  = λf.λx.x           : (τ→τ)→τ→τ")
print("  ONE   = λf.λx.(f x)       : (τ→τ)→τ→τ")
print("  TWO   = λf.λx.(f (f x))   : (τ→τ)→τ→τ")
print("\nChurch Booleans:")
print("  TRUE  = λx.λy.x           : τ₁→τ₂→τ₁")
print("  FALSE = λx.λy.y           : τ₁→τ₂→τ₂")

## 3. Arrow Types: Functions Have Types Too

If we know the types of inputs and outputs, we can describe the type of a **function**:

$$\tau_1 \to \tau_2$$

This reads: "a passage that accepts $\tau_1$ and produces $\tau_2$"

Examples:
- `nat → nat` — a function from naturals to naturals (like successor)
- `bool → nat` — a function from booleans to naturals
- `(nat → nat) → nat → nat` — a function that takes a function and returns a function

In [ ]:
# Examples of arrow types
print("Arrow Type Examples:")
print()
print("  Identity on nat:")
print("    λx.x : nat → nat")
print("    'Takes a natural, returns a natural'")
print()
print("  Successor:")
print("    λn.λf.λx.(f ((n f) x)) : Nat → Nat")
print("    where Nat = (τ→τ)→τ→τ")
print()
print("  Composition:")
print("    λf.λg.λx.(f (g x)) : (τ₂→τ₃) → (τ₁→τ₂) → τ₁ → τ₃")
print("    'Takes two functions, returns their composition'")

## 4. The Type Grammar

Types are built recursively:

$$\tau ::= \text{nat} \mid \text{bool} \mid \tau_1 \to \tau_2$$

This says:
- A type can be `nat`
- A type can be `bool`
- If $\tau_1$ and $\tau_2$ are types, then $\tau_1 \to \tau_2$ is also a type

The arrow is **right-associative**: $\tau_1 \to \tau_2 \to \tau_3$ means $\tau_1 \to (\tau_2 \to \tau_3)$

In [ ]:
# Parse and display type structure
def explain_type(type_str):
    """Explain a type in plain language."""
    if type_str == "nat":
        return "a natural number"
    elif type_str == "bool":
        return "a boolean"
    elif "→" in type_str:
        parts = type_str.split("→", 1)
        if len(parts) == 2:
            return f"a function from ({parts[0].strip()}) to ({parts[1].strip()})"
    return type_str

# Show some expressions with their types
typed_exprs = expressions_df[expressions_df['typable'] == True].head(10)

print("Expressions and their types:")
for _, row in typed_exprs.iterrows():
    print(f"\n  {row['expression'][:30]}")
    print(f"    Type: {row['type']}")

## 5. Why Types Prevent Self-Application

Here's Brennis's crucial insight:

For $(x\ x)$ to be valid, we need:
1. $x$ has some type $\tau$
2. $x$ is applied to $x$, so $\tau$ must be a function type: $\tau = \sigma \to \rho$
3. The argument to $x$ is also $x$, so the input type must be $\tau$: $\sigma = \tau$

Combining: $\tau = \tau \to \rho$

This means:
$$\tau = \tau \to \rho = (\tau \to \rho) \to \rho = ((\tau \to \rho) \to \rho) \to \rho = \ldots$$

**No finite type satisfies this equation!**

In [ ]:
# Demonstrate the infinite type problem
print("Why (λx.x x) cannot be typed:")
print()
print("  Assume x has type τ")
print("  In (x x), x is applied to x")
print("  So x must have type τ→σ (it's a function)")
print("  But x is also the argument, so τ = τ→σ")
print()
print("  Expanding τ = τ→σ:")
print("    τ = τ→σ")
print("    τ = (τ→σ)→σ")
print("    τ = ((τ→σ)→σ)→σ")
print("    ... infinite expansion")
print()
print("  No finite type τ exists!")
print("  Therefore (λx.x x) is UNTYPABLE.")

## 6. Omega is Untypable

Since $(\lambda x. x\ x)$ is untypable, Omega is also untypable:

$$\Omega = (\lambda x. x\ x)(\lambda x. x\ x)$$

If the function is untypable, the whole expression is untypable.

**Types prevent Omega by construction!**

In [ ]:
# Show that all non-terminating expressions are untypable
print("Non-terminating expressions vs typability:")
print()

for _, row in expressions_df.iterrows():
    if row['terminates'] == False:
        typable_str = "TYPABLE" if row['typable'] else "UNTYPABLE"
        print(f"  {row['expression'][:40]}...")
        print(f"    Terminates: {row['terminates']}")
        print(f"    Typable: {typable_str}")
        print()

## 7. Type Annotations

In the typed passage calculus, we can annotate lambda abstractions with types:

$$\lambda x:\tau. M$$

This declares that $x$ has type $\tau$ in the body $M$.

Examples:
- $\lambda x:\text{nat}. x$ — identity on naturals
- $\lambda f:(\text{nat}\to\text{nat}). \lambda x:\text{nat}. f\ x$ — apply a function

In [ ]:
# Examples of annotated types
annotated = expressions_df[expressions_df['expression'].str.contains(':', na=False)]

print("Annotated expressions:")
for _, row in annotated.iterrows():
    print(f"\n  {row['expression']}")
    print(f"  Type: {row['type']}")
    print(f"  Note: {row['notes']}")

## 8. The Tradeoff Preview

Types prevent self-application. This is good — no Omega!

But it's also limiting:

- The identity function $\lambda x.x$ must be given a **specific** type
- We can't write a single identity that works for all types
- The Y combinator (needed for recursion) is untypable!

We'll explore this tradeoff in Tutorial 6.

In [ ]:
# Preview the tradeoff
tradeoff = comparison_df[comparison_df['expression_name'].isin([
    'Identity', 'Universal polymorphic id', 'Y combinator', 'Factorial (with Y)', 'Factorial (primitive rec)'
])]

print("The Safety/Expressiveness Tradeoff:")
for _, row in tradeoff.iterrows():
    typable = "YES" if row['typable'] == True else ("PARTIAL" if row['typable'] == 'PARTIAL' else "NO")
    print(f"\n  {row['expression_name']}:")
    print(f"    Typable: {typable}")
    print(f"    Note: {row['expressiveness_note']}")

## Summary

In this tutorial, we learned:

1. **Base types** (`nat`, `bool`) are atomic classifications
2. **Arrow types** ($\tau_1 \to \tau_2$) describe functions
3. Types are built recursively: $\tau ::= \text{nat} \mid \text{bool} \mid \tau_1 \to \tau_2$
4. **Self-application** requires $\tau = \tau \to \sigma$ — no finite type works
5. **Omega is untypable** — types prevent non-termination structurally
6. There's a **tradeoff**: types prevent dangerous patterns but also limit expressiveness

In the next tutorial, we'll learn the **typing rules** that determine which expressions have types.

---

## Exercises

1. What is the type of $\lambda x:\text{nat}.\lambda y:\text{nat}. x$?

2. What is the type of $\lambda f:(\text{nat}\to\text{bool}).\lambda n:\text{nat}. f\ n$?

3. Why can't $(\lambda x.x\ x\ x)$ be typed? (Hint: what equation would $\tau$ need to satisfy?)

4. Is $\lambda x.\lambda y. x\ y\ y$ typable? If so, what's its type?

---

*Next: Tutorial 3 - The Typing Rules*